# PETSc training getting down to the cool kids

## Quick start
First install PETSc

In [1]:
!conda install -c conda-forge petsc4py

Solving environment: done

# All requested packages already installed.



## Laplace equation

This example solves the Laplace equation,
$$-\Delta u = 1$$
using CG/Jacobi. This illustrates the use of custom Python operators, solvers, and preconditioners.

We start by importing the Python PETSc module and initialising. We are also importing example100 that has some utility stuff.

In [3]:
import petsc4py
from petsc4py import PETSc
import example100

petsc4py.init()

OptDB = PETSc.Options()
N     = OptDB.getInt('N', 100)
draw  = OptDB.getBool('draw', False)

# Setting up the sparse matrix

In [4]:
A = PETSc.Mat()
A.create(comm=PETSc.COMM_WORLD)
A.setSizes([N,N])
A.setType(PETSc.Mat.Type.PYTHON)
A.setPythonContext(example100.Laplace1D())
A.setUp()

In [5]:
x, b = A.getVecs()
b.set(1)

ksp = PETSc.KSP()
ksp.create(comm=PETSc.COMM_WORLD)
ksp.setType(PETSc.KSP.Type.PYTHON)
ksp.setPythonContext(example100.ConjGrad())

pc = ksp.getPC()
pc.setType(PETSc.PC.Type.PYTHON)
pc.setPythonContext(example100.Jacobi())

ksp.setOperators(A, A)
ksp.setFromOptions()
ksp.solve(b, x)

r = b.duplicate()
A.mult(x, r)
r.aypx(-1, b)
rnorm = r.norm()
print(rnorm)
PETSc.Sys.Print('error norm = %g' % rnorm, comm=PETSc.COMM_WORLD)

if draw:
    viewer = PETSc.Viewer.DRAW(x.getComm())
    x.view(viewer)
    PETSc.Sys.sleep(2)

1.1044800808975179e-12
